In [1]:
import pandas as pd
import numpy as np

Import RAW csv file downloaded from: https://www.kaggle.com/kingburrito666/shakespeare-plays

In [2]:
df = pd.read_csv('../data/raw/Shakespeare_data.csv')

Convert the 'Player' Column to string so we can remove the nan entries.
Replace non-alpha-numeric characters in the character lines to aggregate
words together such as 'You,' and 'You'

In [3]:
df['Player'] = df['Player'].astype(str).str.replace('[^a-zA-Z ]', '')
df['PlayerLine'] = df['PlayerLine'].astype(str).str.replace('[^a-zA-Z ]', '')
df['ActSceneLine'] = df['ActSceneLine'].astype(str)

Generate a Cleaned_Dataframe.csv for use in future notebooks to prevent copy and pasting of code

In [4]:
df = df[df.Player != 'nan']
df['Player'] = df['Player'].replace(['OF SYRACUSE'], 'DROMIO OF SYRACUSE')
df['Player'] = df['Player'].replace(['OF EPHESUS'], 'DROMIO OF EPHESUS')
df['Player'] = df['Player'].replace(['OF WINCHESTER'], 'BISHOP OF WINCHESTER')
df['PlayerLine'] = df['PlayerLine'].str.split(' ',expand=True)
df[['Act','Scene','Line']] = df.ActSceneLine.str.split(pat='.',expand=True)
df = df.drop('ActSceneLine',1)
df.to_csv('../data/processed/Cleaned_Dataframe.csv')

Group the Dataframe by the Play and the Player and then save the number of rows related to that grouping.
The grouping of Play and Player with size can be used to remove the lower 3/4 of players who have less than
the upper 1/4 number of lines in a play. 3/4 of the possible classifications will be removed but a majority of
which was comprised of one-off characters such as Murderer or Courier.

In [5]:
df_lines_by_player = df.groupby(['Play','Player']).size()

In [6]:
valid_players = df_lines_by_player[df_lines_by_player > df_lines_by_player.quantile(.75)].index.get_level_values('Player').tolist()

In [7]:
df = df[df.Player.isin(valid_players)]

In [8]:
df['PlayerLine'] = df['PlayerLine'].str.split(' ',expand=True)

In [9]:
df_occurence_of_words = df.groupby('Player')['PlayerLine'].value_counts()

In [10]:
df = df_occurence_of_words.to_frame()

In [11]:
df = df[df.PlayerLine > 1]

Export the transformed data that contains Players with Word Counts to a csv
at '../data/processed/Word_Counts.csv'. This CSV will be used in generating
the Player prediction model.

In [12]:
df.to_csv('../data/processed/Word_Counts.csv')

In [13]:
df = pd.read_csv('../data/processed/Word_Counts.csv')

In [14]:
df

,Player,PlayerLine,PlayerLine.1
0,AARON,And,49
1,AARON,To,18
2,AARON,That,16
3,AARON,I,13
4,AARON,The,13
...,...,...,...
13875,YORK,Thus,2
13876,YORK,Thy,2
13877,YORK,Whiles,2
13878,YORK,Whom,2
